In [1]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import visualization
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import BertTokenizer, BertModel

/data/AI/backup/namlt/namlt/temp/anaconda3/envs/bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model = BertForSequenceClassification.from_pretrained("jackaduma/SecBERT", num_labels=3).to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained('jackaduma/SecBERT')
# initialize the explanations generator


classifications = ["normal","sqli", "xss"]
model.to('cuda')
explanations = Generator(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jackaduma/SecBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
checkpoint = torch.load('/data/AI/backup/namlt/namlt/Hidden/Transformer-Explainability/sql_xss.pt')

In [4]:
dict = checkpoint['state_dict']
dict1 = dict.copy()
for name in dict.keys():
    name_new = name.replace('bert_model', 'bert')
    dict1[name_new] = dict[name]
    del dict1[name]

In [5]:
dict2 = model.state_dict()
# dict2["bert.embeddings.position_ids"]

In [6]:
dict1["bert.embeddings.position_embeddings.weight"] = dict["bert_model.embeddings.position_embeddings.weight"]
dict1["bert.embeddings.word_embeddings.weight"] = dict["bert_model.embeddings.word_embeddings.weight"]
dict1["classifier.weight"] = dict["linear.weight"]
dict1["classifier.bias"] = dict["linear.bias"]
dict1["bert.embeddings.position_ids"] = dict2["bert.embeddings.position_ids"]

In [9]:
model.load_state_dict(dict1)

<All keys matched successfully>

In [11]:
text_batch = ["\t </span>"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")
token_type_ids = encoding['token_type_ids'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
print(class_name)
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name != "normal":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)


/data/AI/backup/namlt/namlt/temp/anaconda3/envs/bert/lib/python3.10/site-packages/transformers/modeling_utils.py:962: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 154.00 MiB (GPU 0; 22.38 GiB total capacity; 1.87 GiB already allocated; 38.81 MiB free; 1.98 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF